In [1]:
!pip install noise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for noise: filename=noise-1.2.2-cp310-cp310-linux_x86_64.whl size=67500 sha256=a02a4cbb1e3effb8c2e162d1530a6331d08cb2d271e9374014a055d168a63532
  Stored in directory: /root/.cache/pip/wheels/b2/01/64/ca2f6ed556c204d5ce2d3719a58757a5504aeb5525654692d0
Successfully built noise


In [2]:
import numpy as np
from PIL import Image
from noise import pnoise2

In [3]:
def generate_perlin_noise_color_texture(image_size):
    width, height = image_size
    scale = 100
    octaves = 6
    persistence = 0.5
    lacunarity = 2.0
    seed_r = np.random.randint(0, 100)
    seed_g = np.random.randint(0, 100)
    seed_b = np.random.randint(0, 100)

    texture = np.zeros((height, width, 3), dtype=np.float32)

    for y in range(height):
        for x in range(width):
            nx, ny = x / scale, y / scale
            value_r = pnoise2(nx, ny, octaves=octaves, persistence=persistence, lacunarity=lacunarity, repeatx=1024, repeaty=1024, base=seed_r)
            value_g = pnoise2(nx, ny, octaves=octaves, persistence=persistence, lacunarity=lacunarity, repeatx=1024, repeaty=1024, base=seed_g)
            value_b = pnoise2(nx, ny, octaves=octaves, persistence=persistence, lacunarity=lacunarity, repeatx=1024, repeaty=1024, base=seed_b)
            texture[y, x, 0] = value_r
            texture[y, x, 1] = value_g
            texture[y, x, 2] = value_b

    min_value, max_value = np.min(texture), np.max(texture)
    normalized_texture = ((texture - min_value) / (max_value - min_value) * 255).astype(np.uint8)

    return Image.fromarray(normalized_texture, mode='RGB')

In [4]:
def generate_sine_wave_color_texture(image_size):
    width, height = image_size
    texture = np.zeros((height, width, 3), dtype=np.float32)

    frequency_r = np.random.uniform(0.01, 0.1)
    frequency_g = np.random.uniform(0.01, 0.1)
    frequency_b = np.random.uniform(0.01, 0.1)

    for y in range(height):
        for x in range(width):
            value_r = 0.5 * (1 + np.sin(2 * np.pi * frequency_r * x))
            value_g = 0.5 * (1 + np.sin(2 * np.pi * frequency_g * y))
            value_b = 0.5 * (1 + np.sin(2 * np.pi * frequency_b * (x + y)))
            texture[y, x, 0] = value_r
            texture[y, x, 1] = value_g
            texture[y, x, 2] = value_b

    normalized_texture = (texture * 255).astype(np.uint8)

    return Image.fromarray(normalized_texture, mode='RGB')


In [9]:
def generate_cellular_automata_color_texture(image_size, iterations=100):
    width, height = image_size
    texture = np.random.randint(0, 2, size=(height, width, 3), dtype=np.uint8)

    def step(state):
        kernel = np.array([
            [1, 1, 1],
            [1, 0, 1],
            [1, 1, 1]
        ])

        neighbors_count = np.zeros_like(state)
        for x_shift in range(-1, 2):
            for y_shift in range(-1, 2):
                if x_shift == 0 and y_shift == 0:
                    continue
                neighbors_count += np.roll(np.roll(state, y_shift, axis=0), x_shift, axis=1) * kernel[y_shift + 1, x_shift + 1]

        new_state = np.zeros_like(state)
        new_state[np.logical_or(np.logical_and(state == 1, neighbors_count == 2), neighbors_count == 3)] = 1
        return new_state

    for _ in range(iterations):
        texture[:, :, 0] = step(texture[:, :, 0])
        texture[:, :, 1] = step(texture[:, :, 1])
        texture[:, :, 2] = step(texture[:, :, 2])

    normalized_texture = (texture * 255).astype(np.uint8)

    return Image.fromarray(normalized_texture, mode='RGB')


In [10]:
def generate_mandelbrot_fractal_color_texture(image_size, max_iter=1000):
    width, height = image_size
    texture = np.zeros((height, width, 3), dtype=np.uint8)

    x_min, x_max = -2.0, 1.0
    y_min, y_max = -1.5, 1.5

    x_values = np.linspace(x_min, x_max, width)
    y_values = np.linspace(y_min, y_max, height)

    mandelbrot_set = np.zeros((height, width), dtype=np.float32)

    for y_index, y in enumerate(y_values):
        for x_index, x in enumerate(x_values):
            zx, zy = x, y
            c = zx + zy * 1j
            z = c
            for i in range(max_iter):
                if abs(z) > 2.0:
                    break 
                z = z * z + c
            mandelbrot_set[y_index, x_index] = i / max_iter

    # Use the Mandelbrot set values to create a color texture.
    texture[:, :, 0] = (mandelbrot_set * 255).astype(np.uint8)
    texture[:, :, 1] = (mandelbrot_set * 255 * 0.5).astype(np.uint8)
    texture[:, :, 2] = (mandelbrot_set * 255 * 0.8).astype(np.uint8)

    return Image.fromarray(texture, mode='RGB')


In [11]:
import os
import random

def generate_and_save_random_textures(num_samples, image_size, output_dir='textures', jpeg_quality=90):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    texture_generators = [
        generate_perlin_noise_color_texture,
        generate_sine_wave_color_texture,
        generate_cellular_automata_color_texture,
        generate_mandelbrot_fractal_color_texture
    ]

    weighted_texture_generators = (
        [texture_generators[0]] * 96 +
        [texture_generators[1]] * 96 +
        [texture_generators[2]] * 96 +
        [texture_generators[3]] * 4
    )

    for i in range(num_samples):
        texture_generator = random.choice(weighted_texture_generators)
        image = texture_generator(image_size)
        image.save(os.path.join(output_dir, f'texture_{i+1}.jpg'), format='JPEG', quality=jpeg_quality)




In [ ]:
num_samples = 100
image_size = (512, 512)
generate_and_save_random_textures(num_samples, image_size)

In [8]:
from PIL import Image

# Create a new image with size 512x512 pixels and all black pixels
black_image = Image.new("RGB", (512, 512), color="black")
# Save the black image as a PNG file in the "texture" folder
black_image.save(os.path.join('textures', f'texture_10.jpg'), format='JPEG', quality=90)

# Create a new image with size 512x512 pixels and all white pixels
white_image = Image.new("RGB", (512, 512), color="white")
# Save the white image as a PNG file in the "texture" folder
white_image.save(os.path.join('textures', f'texture_25.jpg'), format='JPEG', quality=90)


In [ ]:
!zip -r textures.zip /content/textures

updating: content/textures/ (stored 0%)
updating: content/textures/texture_61.jpg (deflated 0%)
updating: content/textures/texture_30.jpg (deflated 1%)
updating: content/textures/texture_2.jpg (deflated 1%)
updating: content/textures/texture_11.jpg (deflated 0%)
updating: content/textures/texture_8.jpg (deflated 1%)
updating: content/textures/texture_63.jpg (deflated 1%)
updating: content/textures/texture_38.jpg (deflated 3%)
updating: content/textures/texture_85.jpg (deflated 1%)
updating: content/textures/texture_43.jpg (deflated 1%)
updating: content/textures/texture_75.jpg (deflated 0%)
updating: content/textures/texture_76.jpg (deflated 9%)
updating: content/textures/texture_24.jpg (deflated 1%)
updating: content/textures/texture_98.jpg (deflated 0%)
updating: content/textures/texture_46.jpg (deflated 0%)
updating: content/textures/texture_5.jpg (deflated 8%)
updating: content/textures/texture_81.jpg (deflated 6%)
updating: content/textures/texture_74.jpg (deflated 1%)
updating: c